## Training the model

In [ ]:
import tensorflow as tf
import collections
#from tensorflow import keras
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPool2D,GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
#inputshape=(height,width,deep)

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import h5py
import os 
from PIL import Image
import sklearn
from sklearn import preprocessing
import scipy
import numpy as np
import PIL.ImageOps
import random

In [ ]:
train_dataset = h5py.File("numbers_dataset_20200720-6.h5",'r')
X_train = np.array(train_dataset['x_train'][:])
Y_train = np.array(train_dataset['y_train'][:])
X_test = np.array(train_dataset['x_test'][:])
Y_test = np.array(train_dataset['y_test'][:])
train_dataset.close()

In [ ]:
class_names=["one","two","three","four","five","six","seven","eight"]
print(len(X_train))
for i in range(64):
    print(np.squeeze(X_train[i]),np.sum(X_train[i]==1))
    print(class_names[int(Y_train[i])])

In [ ]:
print("number of traning examples = "+str(X_train.shape[0]))
print("number of test examples = "+str(X_test.shape[0]))
print("X_train shape:" + str(X_train.shape))
print("Y_train shape:" + str(Y_train.shape))
print("X_test shape:" + str(X_test.shape))
print("Y_test shape:" + str(Y_test.shape))
train_data=(X_train,Y_train)
test_data=(X_test,Y_test)

In [ ]:
#Build a model
def Pimodel(input_shape):
    X_input = Input(shape=input_shape)
    X=ZeroPadding2D(padding=(1,1))(X_input)
    X=Conv2D(1,kernel_size=(3,6),strides=(3,6),use_bias=None)(X)
    X=Activation('relu')(X)
    #X=Conv2D(1,kernel_size=(2,3),strides=(2,2))(X)
    #X=Activation("sigmoid")(X)
    X=Flatten()(X)
    Y=Activation('softmax')(X)
    model=Model(inputs=X_input,outputs=Y,name="JSmodel")
    return model

In [ ]:
picmodel = Pimodel((10,10,1))

In [ ]:
import keras
picmodel.compile(optimizer=keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
picmodel.summary()

In [ ]:
history=picmodel.fit(x=X_train,y=Y_train,batch_size=80,epochs=150,validation_data=(X_test,Y_test))

In [ ]:
preds = picmodel.evaluate(x=X_test,y=Y_test)

print('Test Loss=' + str(preds[0]))
print("Test Accuracy ="+str(preds[1]))

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim((0, 1)) # Uncomment this when showing you model for pay raise
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("accuary.png")

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
arr=np.load("./10dataset/eight/8-0.npy")
x=np.expand_dims(arr,axis=0)
x=np.expand_dims(x,axis=3)
print(picmodel.predict(x))
class_names[np.argmax(picmodel.predict(x))]

## Check the weight and bias

In [ ]:
picmodel = load_model("./numbers_model_20200714.h5")

In [ ]:
from keras import backend as K
from keras.models import load_model
import numpy as np
def get_layer_output(model,x,index=-1):
    layer = K.function([model.input],[model.layers[index].output])
    return layer([x])[0]
input_y=np.load("./dataset3/four/4-10952.npy")
input_x = np.expand_dims(input_y,axis=0)
input_x = np.expand_dims(input_x,axis=3)
layer_0 = get_layer_output(picmodel,input_x,index=0)
layer_1 = get_layer_output(picmodel,input_x,index=1)
layer_2 = get_layer_output(picmodel,input_x,index=2)
layer_3 = get_layer_output(picmodel,input_x,index=3)
layer_4 = get_layer_output(picmodel,input_x,index=4)
layer_5 = get_layer_output(picmodel,input_x,index=5)
print(np.squeeze(layer_0))
print(np.squeeze(layer_1))
print(np.squeeze(layer_2))
print(np.squeeze(layer_3))
print(np.squeeze(layer_4))
print(np.squeeze(layer_5))
class_names[np.argmax(picmodel.predict(x))]

In [ ]:
kernel_weights=np.squeeze(picmodel.get_weights())
#print(kernel_weights.flatten())
a1=np.squeeze(kernel_weights)[0:3,0:6].flatten()
print(kernel_weights)
N=abs(a1.flatten())[np.argmax(abs(a1.flatten()))]
print(N)
a1_nor=a1.flatten()/N
print(a1_nor)

In [ ]:
picmodel.summary()

## the layer output 

In [ ]:
from keras import backend as K
from keras.models import load_model
import numpy
import random
import os,sys
def get_layer_output(model,x,index=0):
    layer = K.function([model.input],[model.layers[index].output])
    return layer([x])[0]
path="./dataset3/one/"
imlist=os.listdir(path)
#rlist=random.sample(imlist,1000)
one_t=[]
One_t=[]
fn_t=[]
for elef in imlist:
    """
    im=Image.fromarray(np.load("./dataset/one/"+elef))
    im=PIL.ImageOps.invert(im)
    im2=im.resize((10,10))
    L=np.array(im2).flatten()
    for i in range(100):
        if L[i]>200:
            L[i]=0
        else:
            L[i]=1
    """
    input_y=np.load("./dataset3/one/"+elef)
    input_x = np.expand_dims(input_y,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    one_t.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="one":
        FN_t=[]
        FN_t.append(elef)
        FN_t.append(layer_2.flatten())
        fn_t.append(FN_t)
        print(elef,layer_4,class_names[np.argmax(layer_5)])
        One_t.append(elef) 
one1=collections.Counter(one_t)
print(len(one_t),collections.Counter(one_t))

In [ ]:
path="./dataset2/two/"
imlist=os.listdir(path)
#rlist=random.sample(imlist,1000)
two_t=[]
Two_t=[]
for elef in imlist:
    """
    im=Image.fromarray(np.load("./newdataset/two/"+elef))
    im=PIL.ImageOps.invert(im)
    im2=im.resize((10,10))
    L=np.array(im2).flatten()
    for i in range(100):
        if L[i]>200:
            L[i]=0
        else:
            L[i]=1
    input_y=np.array(L).reshape(10,10)
    """
    input_y=np.load("./dataset2/two/"+elef)
    input_x = np.expand_dims(input_y,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    two_t.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="two":
        print(elef,layer_4,class_names[np.argmax(layer_5)])
        Two_t.append(elef)
two1=collections.Counter(two_t)
print(len(two_t),collections.Counter(two_t))

In [ ]:
path="./dataset3/three/"
imlist=os.listdir(path)
#rmlist=random.sample(imlist,1000)
three_t=[]
Three_t=[]
for elef in imlist:
    """
    im=Image.fromarray(np.load("./newdataset/three/"+elef))
    im=PIL.ImageOps.invert(im)
    im2=im.resize((10,10))
    L=np.array(im2).flatten()
    for i in range(100):
        if L[i]>200:
            L[i]=0
        else:
            L[i]=1
    input_y=np.array(L).reshape(10,10)
    """
    input_y=np.load("./dataset3/three/"+elef)
    input_x = np.expand_dims(input_y,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    three_t.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="three":
        print(elef,layer_4,class_names[np.argmax(layer_5)])
        Three_t.append(elef)
three1=collections.Counter(three_t)
print(len(three_t),collections.Counter(three_t))

In [ ]:
path="./dataset3/four/"
imlist=os.listdir(path)
#rmlist=random.sample(imlist,1000)
four_t=[]
Four_t=[]
for elef in imlist:
    """
    im=Image.fromarray(np.load("./newdataset/four/"+elef))
    im=PIL.ImageOps.invert(im)
    im2=im.resize((10,10))
    L=np.array(im2).flatten()
    for i in range(100):
        if L[i]>200:
            L[i]=0
        else:
            L[i]=1
    input_y=np.array(L).reshape(10,10)
    """
    input_y=np.load("./dataset3/four/"+elef)
    input_x = np.expand_dims(input_y,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    four_t.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="four":
        print(elef,layer_4,class_names[np.argmax(layer_5)])
        Four_t.append(elef)
four1=collections.Counter(four_t)
print(len(four_t),collections.Counter(four_t))

In [ ]:
path="./dataset3/five/"
imlist=os.listdir(path)
#rmlist=random.sample(imlist,1000)
five_t=[]
Five_t=[]
for elef in imlist:
    """
    im=Image.fromarray(np.load("./newdataset/five/"+elef))
    im=PIL.ImageOps.invert(im)
    im2=im.resize((10,10))
    L=np.array(im2).flatten()
    for i in range(100):
        if L[i]>200:
            L[i]=0
        else:
            L[i]=1
    input_y=np.array(L).reshape(10,10)
    """
    input_y=np.load("./dataset3/five/"+elef)
    input_x = np.expand_dims(input_y,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    five_t.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="five":
        print(elef,layer_4,class_names[np.argmax(layer_5)])
        Five_t.append(elef)
five1=collections.Counter(five_t)
print(len(five_t),collections.Counter(five_t))

In [ ]:
path="./dataset3/six/"
imlist=os.listdir(path)
#rmlist=random.sample(imlist,1000)
six_t=[]
Six_t=[]
for elef in imlist:
    """
    im=Image.fromarray(np.load("./newdataset/six/"+elef))
    im=PIL.ImageOps.invert(im)
    im2=im.resize((10,10))
    L=np.array(im2).flatten()
    for i in range(100):
        if L[i]>200:
            L[i]=0
        else:
            L[i]=1
    input_y=np.array(L).reshape(10,10)
    """
    input_y=np.load("./dataset3/six/"+elef)
    input_x = np.expand_dims(input_y,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    six_t.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="six":
        print(elef,layer_4,class_names[np.argmax(layer_5)])
        Six_t.append(elef)
six1=collections.Counter(six_t)
print(len(six_t),collections.Counter(six_t))

In [ ]:
path="./dataset3/seven/"
imlist=os.listdir(path)
#rmlist=random.sample(imlist,1000)
seven_t=[]
Seven_t=[]
for elef in imlist:
    """
    im=Image.fromarray(np.load("./newdataset/seven/"+elef))
    im=PIL.ImageOps.invert(im)
    im2=im.resize((10,10))
    L=np.array(im2).flatten()
    for i in range(100):
        if L[i]>200:
            L[i]=0
        else:
            L[i]=1
    input_y=np.array(L).reshape(10,10)
    """
    input_y=np.load("./dataset3/seven/"+elef)
    input_x = np.expand_dims(input_y,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    seven_t.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="seven":
        print(elef,layer_4,class_names[np.argmax(layer_5)])
        Seven_t.append(elef)
seven1=collections.Counter(three_t)
print(len(seven_t),collections.Counter(seven_t))

In [ ]:
path="./dataset3/eight/"
imlist=os.listdir(path)
#rmlist=random.sample(imlist,1000)
eight_t=[]
Eight_t=[]
for elef in imlist:
    """
    im=Image.fromarray(np.load("./dataset/eight/"+elef))
    imt=im.resize((10,10))
    arr=np.array(imt)
    input_y=np.where(arr<1,arr,1)
    """
    input_y=np.load("./dataset3/eight/"+elef)
    input_x = np.expand_dims(input_y,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    eight_t.append(class_names[np.argmax(layer_5)])
    if class_names[np.argmax(layer_5)]=="eight":
        print(elef,layer_4,class_names[np.argmax(layer_5)])
        Eight_t.append(elef)
eight1=collections.Counter(eight_t)
print(len(eight_t),collections.Counter(eight_t))

In [ ]:
import pandas as pd
#dict1 = pd.DataFrame(pd.Series(f1),columns=["numbers"],index=["fire","soil","water","wood","gas","sky","human","life"])
x=["one","two","three","four","five","six","seven","eight"]
dict1=pd.Series(one1).reindex(["one","two","three","four","five","six","seven","eight"])
dict1=dict1.fillna(0)
print(dict1)
print(dict1.fillna(0))
#dict1[np.squeeze(np.argwhere(np.isnan(dict1)))]=0

dict2=pd.Series(two1).reindex(["one","two","three","four","five","six","seven","eight"])
#dict2[np.squeeze(np.argwhere(np.isnan(dict2)))]=0
dict2=dict2.fillna(0)
dict3=pd.Series(three1).reindex(["one","two","three","four","five","six","seven","eight"])
dict3=dict3.fillna(0)
#dict3[np.squeeze(np.argwhere(np.isnan(dict3)))]=0
dict4=pd.Series(four1).reindex(["one","two","three","four","five","six","seven","eight"])
dict4=dict4.fillna(0)
#dict4[np.squeeze(np.argwhere(np.isnan(dict4)))]=0
dict5=pd.Series(five1).reindex(["one","two","three","four","five","six","seven","eight"])
dict5=dict5.fillna(0)
#dict5[np.squeeze(np.argwhere(np.isnan(dict5)))]=0
dict6=pd.Series(six1).reindex(["one","two","three","four","five","six","seven","eight"])
dict6=dict6.fillna(0)
#dict6[np.squeeze(np.argwhere(np.isnan(dict6)))]=0
dict7=pd.Series(seven1).reindex(["one","two","three","four","five","six","seven","eight"])
dict7=dict7.fillna(0)
#dict7[np.squeeze(np.argwhere(np.isnan(dict7)))]=0
dict8=pd.Series(eight1).reindex(["one","two","three","four","five","six","seven","eight"])
dict8=dict8.fillna(0)
#dict8[np.squeeze(np.argwhere(np.isnan(dict8)))]=0
plt.figure(figsize=(24,10))
plt.subplot(241)
plt.bar(x,dict1,color="red")
plt.title("one regonition")
for i,j in zip(x, dict1):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(242)
plt.bar(x,dict2,color="yellow")
plt.title("two regonition")
for i,j in zip(x, dict2):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict2)
plt.subplot(243)
plt.bar(x,dict3,color="brown")
plt.title("three regonition")
for i,j in zip(x, dict3):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(244)
plt.bar(x,dict4,color="blue")
plt.title("four regonition")
for i,j in zip(x, dict4):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(245)
plt.bar(x,dict5,color="green")
plt.title("five regonition")
for i,j in zip(x, dict5):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(246)
plt.bar(x,dict6,color="orange")
plt.title("six regonition")
for i,j in zip(x, dict6):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(247)
plt.bar(x,dict7,color="black")
plt.title("seven regonition")
for i,j in zip(x, dict7):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
plt.subplot(248)
plt.bar(x,dict8,color="purple")
plt.title("eight regonition")
for i,j in zip(x, dict8):
        plt.text(i, j, str(int(j)), ha='center', va='bottom', fontsize=10.5)
#print(dict1)
#list1=dict1.values.tolist()
plt.savefig("regonition.png")
plt.show()
